In this notebook we will generate the feature data used to train the classification algorithm

In [1]:
# Load the code from Rojas
load("polyB.sage")

# Data from Rojas, Behn on group ID and signature pairs in the 
# from (g, ID, signature)
load("grupos_que_actuan.sage")

ver 2020.07.24


In [2]:
# Initialise the field with two elements
F2 = FiniteField(2)

# define many simple functions that will correspond to the features we want to record. 
def quotient_genus(g, GO, c):
    return (g-1)/GO - sum([1 - 1/ci for ci in c])/2 + 1

def quotient_genus_zero(genus, group_order, signature):
    return bool(quotient_genus(genus, group_order, signature) == 0)

def number_involutions_upto_conjugacy(G):
    return sum([gi.order() == 2 for gi in G.conjugacy_classes_representatives()])
    
def power_two_dividing(order):
    p2 = -1
    while order in ZZ:
        order /= 2
        p2 += 1
    return p2

def number_odds_in_sig(c):
    return sum([ci % 2 for ci in c])

def number_evens_in_sig(c):
    return len(c) - number_odds_in_sig(c)

def number_involutions(G):
    return sum([cc.cardinality() for cc in G.conjugacy_classes() if cc[0].order() == 2])

def group_action_is_large(g, GO):
    return F2(GO > 4*(g-1))

def dimension_of_family(g, GO, c):
    return 3*(quotient_genus(g, GO, c) - 1) + len(c)

def max_ramification_order(c):
    return max(c)

# load some functions
from sage.geometry.hyperplane_arrangement.affine_subspace import AffineSubspace
import csv

In [ ]:
# Loop over the Rojas data
for g in range(2, 21):
    
    ID_sig_list = grupos_que_actuan(g)
    I2g = matrix.identity(2*g, F2)
    
    for ID, cs in ID_sig_list:
    
        GO = ID[0]
        
        # Turn the group ID into a sage permutation group
        Im = gap.Image(gap.IsomorphismPermGroup(gap.SmallGroup(*ID)))
        G = PermutationGroup(gap_group = Im)
        
        # at this stage record all the features that depend on the group only,
        # not the signature
        group_level_data = [GO, group_action_is_large(g, GO), power_two_dividing(GO), 
                            number_involutions(G), number_involutions_upto_conjugacy(G)]

        for c in cs:
            if not quotient_genus_zero(g, GO, c):
                continue
                
            # at this stage record all the features that depend on the signature and group
            signature_level_data = [number_odds_in_sig(c), number_evens_in_sig(c),
                                   max_ramification_order(c), dimension_of_family(g, GO, c)]

            generating_vectors = find_generator_representatives(G, c)
            
            data = []

            # loop over the possible generating vectors for a given genus, group, and signauture. 
            for ngen in range(len(generating_vectors)):
                V = generating_vectors[ngen]
                # Try to get the polyon associated to the action. 
                # If there is an error, as sometimes happens with the code, just print the error and 
                # continue
                try:
                    P = Poly(G, V)
                except ValueError:
                    print("Unexplained error with data {}".format((g, ID, c, ngen)))
                    continue
                
                # now get the intersection matrix to transfer the group action to 
                # a symplectic one
                _, PT = P.intersection_matrix().symplectic_form()
                
                if not PT.nrows()==2*g:
                    print("Unexplained error with data {}".format((g, ID, c, ngen)))
                    continue
                
                Q = PT.transpose().inverse()
                symplectic_group_generators = [Q*P._representation(g)*PT.transpose() for g in P.generators]
                
                # Compute the action of the generators
                Mlist = [matrix(M).change_ring(F2) for M in symplectic_group_generators]

                Vbars = [vector([sum([A[j,i]*A[j+g,i] for j in range(g)]) 
                     for i in range(2*g)], F2) for A in Mlist]
                
                MIs = [M-I2g for M in Mlist]
                X0s = [MI.solve_left(-V) for MI, V in zip(MIs, Vbars)]
                # Find the affine subspace of vectors invariant under the action of each gen
                AS = [AffineSubspace(X0, MI.kernel()) for X0, MI in zip(X0s, MIs)]

                # compute the intersections of these affine subspaces. 
                Sols = AS.pop()
                while Sols and AS:
                    AI = AS.pop()
                    Sols = Sols.intersection(AI)
                
                # record the dimension of the space of invariants, taking -1 if there are 
                # no invariants. 
                if Sols:
                    data.append([g] + group_level_data + signature_level_data + [Sols.dimension()])
                else:
                    data.append([g] + group_level_data + signature_level_data + [-1])

            # write the data into the existing csv
            with open("group_action_features_invariants_data.csv", "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerows(data)

Unexplained error with data (9, [32, 18], [2, 2, 2, 2, 2], 0)
Unexplained error with data (10, [18, 1], [2, 2, 2, 2, 2, 2], 0)
Unexplained error with data (10, [36, 4], [2, 2, 2, 2, 2], 0)


Record the values with unexpected errors.
 - Unexplained error with data (9, [32, 18], [2, 2, 2, 2, 2], 0)
 - Unexplained error with data (10, [18, 1], [2, 2, 2, 2, 2, 2], 0)
 - Unexplained error with data (10, [36, 4], [2, 2, 2, 2, 2], 0)
 - Unexplained error with data (11, [20, 4], [2, 2, 2, 2, 2, 2], 0)
 - Unexplained error with data (11, [40, 6], [2, 2, 2, 2, 2], 0)
 - Unexplained error with data (12, [22, 1], [2, 2, 2, 2, 2, 2], 0)
 - Unexplained error with data (12, [44, 3], [2, 2, 2, 2, 2], 0)
 - Unexplained error with data (13, [2, 1], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], 0)

For future investigations, one might look at the following links to get additional data
 - https://groups.google.com/g/lmfdb-support/c/aoipQ-qgQ6Q/m/HJkzEweWBAAJ
 - https://github.com/LMFDB/lmfdb-inventory/blob/master/db-curve_automorphisms.md
 - http://www.lmfdb.org/api/hgcwa_passports/